### First we set up our configuration

In [3]:
COUCHBASE_SERVER = "localhost"
COUCHBASE_BUCKET = "travel-sample"

import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 200px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

### Next, we import necessary Couchbase libraries

In [4]:
from couchbase.bucket import Bucket
import couchbase.subdocument as SD
from couchbase import exceptions

### Connect to Couchbase

In [5]:
try:
    bucket = Bucket("couchbase://%s/%s" % (COUCHBASE_SERVER,COUCHBASE_BUCKET))
except exceptions.CouchbaseError as err:
    print("Couchbase error: %s" % err)
else:
    print("Successfully connected to Couchbase!")

Successfully connected to Couchbase!


### Retrieve one WHOLE document, by key, from the bucket

In [6]:
keyToRetrieve="airport_1284"
try:
    completeDocument = bucket.get(keyToRetrieve)
except Exception as err:
    if isinstance(err, exceptions.NotFoundError):
        print("Document %s not found in bucket: %s" % (err.key, COUCHBASE_BUCKET))
except:
    print("Unhandled Exception encountered: %s" % err)

### Let's look at the return type of the get operation

In [7]:
print type(completeDocument)

<type 'ValueResult'>


Note that the returned **ValueResult** object from a get command contains also Couchbase Metadata.

### And what it is composed of...

In [8]:
print completeDocument

ValueResult<rc=0x0, key=u'airport_1284', value={u'city': u'Albi', u'tz': u'Europe/Paris', u'country': u'France', u'icao': u'LFCI', u'geo': {u'lat': 43.913887, u'alt': 564.0, u'lon': 2.113056}, u'faa': u'LBI', u'type': u'airport', u'id': 1284, u'airportname': u'Le Sequestre'}, cas=0xfc3052430100, flags=0x2000006>


We want only the **value** property to  get the true document body from Couchbase Server.

### Let's have a look at the document body

In [9]:
print completeDocument.value

{u'city': u'Albi', u'tz': u'Europe/Paris', u'country': u'France', u'icao': u'LFCI', u'geo': {u'lat': 43.913887, u'alt': 564.0, u'lon': 2.113056}, u'faa': u'LBI', u'type': u'airport', u'id': 1284, u'airportname': u'Le Sequestre'}


### So, now that we have seen what the API is like for a full document read, let's look at the sub-document API

In [10]:
fieldToRetrieve="geo"
try:
    subDocument = bucket.lookup_in(keyToRetrieve, SD.get(fieldToRetrieve))
except Exception as err:
    if isinstance(err, exceptions.NotFoundError):
        print("Document %s not found in bucket: %s" % (err.key, COUCHBASE_BUCKET))
except:
    print("Unhandled Exception encountered: %s" % err)

### Let's look at the return type of the subDocument operation

In [11]:
print type(subDocument)

<class 'couchbase.result.SubdocResult'>


### ... and what the return type of the subDocument operation is composed of

In [12]:
print subDocument

SubdocResult<rc=0x0, key='airport_1284', cas=0xfc3052430100, specs=(Spec<GET, 'geo'>,), results=[(0, {u'lat': 43.913887, u'alt': 564.0, u'lon': 2.113056})]>


### We can access the subDocument by using a get command against the result by using the get() method, iteration, or direct index access

#### get()

In [13]:
print subDocument.get("geo")[1]

{u'lat': 43.913887, u'alt': 564.0, u'lon': 2.113056}


#### Iteration

In [14]:
for value in subDocument:
    print value

{u'lat': 43.913887, u'alt': 564.0, u'lon': 2.113056}


#### Index access

In [15]:
print subDocument[0]

{u'lat': 43.913887, u'alt': 564.0, u'lon': 2.113056}
